In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype.interfaces import freesurfer as fsr
from nipype.interfaces import brainsuite
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from shutil import copyfile
import pandas as pd
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [29]:
#Set variables
# subject_list = ['02-T2', '06-T1', '07-T2','08-T1', '09-T2', '10-T1', '11-T1', '12-T1', '13-T1', '15-T1', '16-T1', '17-T1',
#                 '18-T1', '19-T1', '20-T1', '21-T1', '22-T1', '23-T1', '24-T1', '29-T1', '30-T1', '31-T1', '32-T1',
#                 '35-T1', '36-T1', '37-T1', '38-T1', '40-T1', '102-T1', '103-T1', '104-T1', '105-T1', '106-T1', '107-T1', '108-T1', '109-T1',
#                 '110-T1', '111-T1']

subject_list = ['16-T1'] #, '16-T1'] # '02-T2']#, '20-T1','22-T1','29-T1','31-T1','35-T1']
#subject_list = ['123-T1', '124-T1']#, '20-T1','22-T1','29-T1','31-T1','35-T1']

home = '/Users/lucindasisk/Documents/qT1'
fpath = home + '/subjDir'
workflow_dir = home + '/workflow'
data_dir = home + '/data'
data = home + '/flywheel_subjDir'

# subs = glob(data + '/*-*')
# subject_list = pd.Series(subs).str.replace(data + '/', '')
# subject_list

subs = pd.read_csv(home + '/files/Gsheet_QCDONE_finalSubjList_10.22.20.csv', header=0)
#subject_list = subs['ID'].tolist()

#subject_list = ['097-T3']

In [30]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions=[('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(t1 = fpath + '/{subject_id}/T1_acpc.nii.gz',
               qt1 = data + '/{subject_id}/fw_qT1_t1fit_final.nii.gz')

sf = Node(SelectFiles(template, 
                      base_directory = fpath),
          name = 'sf')


In [31]:
#Skullstrip T1w image
stripT1 = Node(fsl.BET(out_file = 'T1_acpc_bet.nii.gz'),
             name = 'stripT1')

#Reorient qT1 to standard orientation
reorient = Node(fsl.Reorient2Std(out_file = 'reoriented_qT1.nii.gz',
                              output_type = 'NIFTI_GZ'),
                    name = 'reorient')

#Register qT1 to T1w
register_qt1 = Node(fsl.FLIRT(out_file = 'registered_qT1_T1acpc.nii.gz',
                              output_type = 'NIFTI_GZ',
                              out_matrix_file = 'flirt_transform_qt1.mat'),
                    name = 'register_qt1')

#Extract voxel values from qT1 beneath overlaid mask - Left Uncinate
get_voxLUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftUF.txt'),
               name = 'get_voxLUF')

#Extract voxel values from qT1 beneath overlaid mask - Right Uncinate
get_voxRUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightUF.txt'),
               name = 'get_voxRUF')

#Extract voxel values from qT1 beneath overlaid mask - Left Cingulum Cingulate
get_voxLCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCC.txt'),
               name = 'get_voxLCC')

#Extract voxel values from qT1 beneath overlaid mask - Right Cingulum Cingulate
get_voxRCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightCC.txt'),
               name = 'get_voxRCC')

#Extract voxel values from qT1 beneath overlaid mask - Left Corticosponal
get_voxLCST = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCST.txt'),
               name = 'get_voxLCST')

#Extract voxel values from qT1 beneath overlaid mask - Right Corticospinal
get_voxRCST = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightCST.txt'),
               name = 'get_voxRCST')

In [21]:
qt1proc_flow = Workflow(name='qt1proc_flow')
qt1proc_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
                      (sf, stripT1, [('t1', 'in_file')]),
                      (stripT1, datasink, [('out_file', '1_check_alignment')]),
                      (sf, reorient, [('qt1', 'in_file')]),
                      (reorient, datasink, [
                       ('out_file', '1_check_alignment.@par')]),
                      (reorient, register_qt1, [('out_file', 'in_file')]),
                      (stripT1, register_qt1, [('out_file', 'reference')]),
                      (register_qt1, datasink, [
                       ('out_file', '1_check_alignment.@par.@par')])
                      ])

qt1proc_flow.base_dir = workflow_dir
qt1proc_flow.write_graph(graph2use='flat')
preproc = qt1proc_flow.run('MultiProc', plugin_args={'n_procs': 4})

201022-12:59:05,417 nipype.workflow DEBUG:
	 (qt1proc_flow.infosource, qt1proc_flow.sf): No edge data
201022-12:59:05,418 nipype.workflow DEBUG:
	 (qt1proc_flow.infosource, qt1proc_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
201022-12:59:05,419 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.stripT1): No edge data
201022-12:59:05,420 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.stripT1): new edge data: {'connect': [('t1', 'in_file')]}
201022-12:59:05,420 nipype.workflow DEBUG:
	 (qt1proc_flow.stripT1, qt1proc_flow.datasink): No edge data
201022-12:59:05,421 nipype.workflow DEBUG:
	 (qt1proc_flow.stripT1, qt1proc_flow.datasink): new edge data: {'connect': [('out_file', '1_check_alignment')]}
201022-12:59:05,421 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.reorient): No edge data
201022-12:59:05,422 nipype.workflow DEBUG:
	 (qt1proc_flow.sf, qt1proc_flow.reorient): new edge data: {'connect': [('qt1', 'in_file')]}
201022-12:59:05,423 

Process ForkProcess-14:
Process ForkProcess-13:
Process ForkProcess-15:
Process ForkProcess-16:
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.7/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/local/anaconda3/lib/pyt

In [32]:
#SelectFiles
template2 = dict(qt1r=data_dir+'/1_check_alignment/{subject_id}/registered_qT1_T1acpc.nii.gz',
               unc_maskL = fpath+'/{subject_id}/Left_Uncinate.nii.gz',
               unc_maskR = fpath+'/{subject_id}/Right_Uncinate.nii.gz',
               cc_maskL = fpath+'/{subject_id}/Left_Cingulum_Cingulate.nii.gz',
               cc_maskR = fpath+'/{subject_id}/Right_Cingulum_Cingulate.nii.gz',
               cst_maskL = fpath+'/{subject_id}/Left_Corticospinal.nii.gz',
               cst_maskR = fpath+'/{subject_id}/Right_Corticospinal.nii.gz')

# 
#             bval=fpath+'/{subject_id}/dti.bval',
#             bvec=fpath+'/{subject_id}/dti.bvec'

sf2 = Node(SelectFiles(template2, 
                      base_directory = fpath),
          name = 'sf2')

In [33]:
get_stats=Workflow(name='get_stats')
get_stats.connect([(infosource, sf2, [('subject_id','subject_id')]),
                   #Left UF
                   (sf2, get_voxLUF, [('qt1r', 'in_file')]),
                   (sf2, get_voxLUF, [('unc_maskL', 'mask')]),
                   (get_voxLUF, datasink, [('out_file', '2_Extracted_Stats')]),
                   #Left Cingulum
                   (sf2, get_voxLCC, [('qt1r', 'in_file')]),
                   (sf2, get_voxLCC, [('cc_maskL', 'mask')]),
                   (get_voxLCC, datasink, [('out_file', '2_Extracted_Stats.@par')]),
                   #Left Corticospinal
                   (sf2, get_voxLCST, [('qt1r', 'in_file')]),
                   (sf2, get_voxLCST, [('cst_maskL', 'mask')]),
                   (get_voxLCST, datasink, [('out_file', '2_Extracted_Stats.@par.@par')]),
                   #Right UF
                   (sf2, get_voxRUF, [('qt1r', 'in_file')]),
                   (sf2, get_voxRUF, [('unc_maskR', 'mask')]),
                   (get_voxRUF, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par')]),
                   #Right Cingulum
                   (sf2, get_voxRCC, [('qt1r', 'in_file')]),
                   (sf2, get_voxRCC, [('cc_maskR', 'mask')]),
                   (get_voxRCC, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par')]),
                   #Right Corticospinal
                   (sf2, get_voxRCST, [('qt1r', 'in_file')]),
                   (sf2, get_voxRCST, [('cst_maskR', 'mask')]),
                   (get_voxRCST, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par')])
                  ])
get_stats.base_dir = workflow_dir
get_stats.write_graph(graph2use = 'flat')
preproc = get_stats.run('MultiProc', plugin_args={'n_procs': 4})

201022-15:18:13,448 nipype.workflow DEBUG:
	 (get_stats.infosource, get_stats.sf2): No edge data
201022-15:18:13,449 nipype.workflow DEBUG:
	 (get_stats.infosource, get_stats.sf2): new edge data: {'connect': [('subject_id', 'subject_id')]}
201022-15:18:13,450 nipype.workflow DEBUG:
	 (get_stats.sf2, get_stats.get_voxLUF): No edge data
201022-15:18:13,450 nipype.workflow DEBUG:
	 (get_stats.sf2, get_stats.get_voxLUF): new edge data: {'connect': [('qt1r', 'in_file')]}
201022-15:18:13,451 nipype.workflow DEBUG:
	 (get_stats.sf2, get_stats.get_voxLUF): Edge data exists: {'connect': [('qt1r', 'in_file')]}
201022-15:18:13,452 nipype.workflow DEBUG:
	 (get_stats.sf2, get_stats.get_voxLUF): new edge data: {'connect': [('qt1r', 'in_file'), ('unc_maskL', 'mask')]}
201022-15:18:13,453 nipype.workflow DEBUG:
	 (get_stats.get_voxLUF, get_stats.datasink): No edge data
201022-15:18:13,454 nipype.workflow DEBUG:
	 (get_stats.get_voxLUF, get_stats.datasink): new edge data: {'connect': [('out_file', '2_

In [ ]:
#SelectFiles
template3 = dict(t1w = fpath+'/{subject_id}/freesurfer_brain.nii.gz',
                aseg = fpath+'/{subject_id}/aparc.a2009s+aseg.nii.gz',
                qt1_reor = data_dir+'/1_check_alignment/_subject_id_{subject_id}/reoriented_qT1.nii.gz')       


sf3 = Node(SelectFiles(template3, 
                      base_directory = fpath),
          name = 'sf3')

In [ ]:
# #MRI Convert Nodes
# convert_wb = Node(fsr.MRIConvert(in_type='mgz',
#                                out_file = 'freesurfer_brain.nii.gz'),
#                name = 'convert_wb')

# convert_aseg = Node(fsr.MRIConvert(in_type='mgz',
#                                out_file = 'aparc.a2009s+aseg.nii.gz'),
#                name = 'convert_aseg')


In [ ]:
#Extract left hippocampus - seg value 17
extract_lhipp = Node(fsl.Threshold(thresh = 16.5, 
                                   direction = 'below', 
                                   args = '-uthr 17.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_hippocampus.nii.gz'),
                        name = 'extract_lhipp')
#Extract right hippocampus - seg value 53
extract_rhipp = Node(fsl.Threshold(thresh = 52.5, 
                                   direction = 'below', 
                                   args = '-uthr 53.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_hippocampus.nii.gz'),
                    name='extract_rhipp')


In [ ]:
#Extract left amygdala - seg value 18
extract_lamyg = Node(fsl.Threshold(thresh = 17.5, 
                                   direction = 'below', 
                                   args = '-uthr 18.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_amygdala.nii.gz'),
                        name = 'extract_lamyg')
#Extract right amygdala - seg value 54
extract_ramyg = Node(fsl.Threshold(thresh = 53.5, 
                                   direction = 'below', 
                                   args = '-uthr 54.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_amygdala.nii.gz'),
                    name='extract_ramyg')

In [ ]:
#Extract left caudate - seg value 11
extract_lcaud = Node(fsl.Threshold(thresh = 10.5, 
                                   direction = 'below', 
                                   args = '-uthr 11.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_caudate.nii.gz'),
                        name = 'extract_lcaud')

#Extract right caudate - seg value 50
extract_rcaud = Node(fsl.Threshold(thresh = 49.5, 
                                   direction = 'below', 
                                   args = '-uthr 50.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_caudate.nii.gz'),
                    name='extract_rcaud')


In [ ]:
#Extract left nucleus accumbens - seg value 26
extract_lnacc  = Node(fsl.Threshold(thresh = 25.5, 
                                   direction = 'below', 
                                   args = '-uthr 26.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_nAcc.nii.gz'),
                        name = 'extract_lnacc')

#Extract right caudate - seg value 58
extract_rnacc = Node(fsl.Threshold(thresh = 57.5, 
                                   direction = 'below', 
                                   args = '-uthr 58.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_nAcc.nii.gz'),
                    name='extract_rnacc')


In [ ]:
#Extract left putamen - seg value 26
extract_lput  = Node(fsl.Threshold(thresh = 11.5, 
                                   direction = 'below', 
                                   args = '-uthr 12.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_putamen.nii.gz'),
                        name = 'extract_lput')

#Extract right caudate - seg value 51
extract_rput = Node(fsl.Threshold(thresh = 50.5, 
                                   direction = 'below', 
                                   args = '-uthr 51.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_putamen.nii.gz'),
                    name='extract_rput')

In [ ]:
#Extract voxel values from qT1 beneath overlaid mask - Left Hippocampus
get_voxRhipp = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightHippocampus.txt'),
               name = 'get_voxRhipp')

#Extract voxel values from qT1 beneath overlaid mask - Right Hippocampus
get_voxLhipp = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftHippocampus.txt'),
               name = 'get_voxLhipp')

In [ ]:
#Extract voxel values from qT1 beneath overlaid mask - Left Amygdala
get_voxRamyg = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightAmygdala.txt'),
               name = 'get_voxRamyg')

#Extract voxel values from qT1 beneath overlaid mask - Right Amygdala
get_voxLamyg = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftAmygdala.txt'),
               name = 'get_voxLamyg')

In [ ]:
#Extract voxel values from qT1 beneath overlaid mask - Left Putamen
get_voxRput = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightPutamen.txt'),
               name = 'get_voxRput')

#Extract voxel values from qT1 beneath overlaid mask - Right Putamen
get_voxLput = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftPutamen.txt'),
               name = 'get_voxLput')

In [ ]:
#Extract voxel values from qT1 beneath overlaid mask - Left NAcc
get_voxRnacc = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightNAcc.txt'),
               name = 'get_voxRnacc')

#Extract voxel values from qT1 beneath overlaid mask - Right NAcc
get_voxLnacc = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftNAcc.txt'),
               name = 'get_voxLnacc')

In [ ]:
#Extract voxel values from qT1 beneath overlaid mask - Left Caudate
get_voxRcaud = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightCaudate.txt'),
               name = 'get_voxRcaud')

#Extract voxel values from qT1 beneath overlaid mask - Right Caudate
get_voxLcaud = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCaudate.txt'),
               name = 'get_voxLcaud')

In [ ]:
#Register qT1 to T1w
register_qt12 = Node(fsl.FLIRT(out_file = 'registered_qT1_toFreesurfer.nii.gz',
                              output_type = 'NIFTI_GZ',
                              out_matrix_file = 'flirt_transform_qt1.mat'),
                    name = 'register_qt12')

In [ ]:
hippdata_flow = Workflow(name='hippdata_flow')
hippdata_flow.connect([(infosource, sf3, [('subject_id', 'subject_id')]),
                       (sf3, register_qt12, [('qt1_reor', 'in_file')]),
                       (sf3, register_qt12, [('t1w', 'reference')]),
                       (sf3, extract_lhipp, [('aseg', 'in_file')]),
                       (sf3, extract_rhipp, [('aseg', 'in_file')]),
                       (sf3, extract_lamyg, [('aseg', 'in_file')]),
                       (sf3, extract_ramyg, [('aseg', 'in_file')]),
                       (sf3, extract_lput, [('aseg', 'in_file')]),
                       (sf3, extract_rput, [('aseg', 'in_file')]),
                       (sf3, extract_lnacc, [('aseg', 'in_file')]),
                       (sf3, extract_rnacc, [('aseg', 'in_file')]),
                       (sf3, extract_lcaud, [('aseg', 'in_file')]),
                       (sf3, extract_rcaud, [('aseg', 'in_file')]),
                       (extract_lhipp, datasink, [('out_file', '1_check_alignment')]),
                       (extract_rhipp, datasink, [('out_file', '1_check_alignment.@par')]),
                       (extract_lamyg, datasink, [('out_file', '1_check_alignment.@par.@par')]),
                       (extract_ramyg, datasink, [('out_file', '1_check_alignment.@par.@par.@par')]),
                       (extract_lput, datasink, [('out_file', '1_check_alignment.@par.@par.@par.@par')]),
                       (extract_rput, datasink, [('out_file', '1_check_alignment.@par.@par.@par.@par.@par')]),
                       (extract_lnacc, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par')]),
                       (extract_rnacc, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par')]),
                       (extract_lcaud, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (extract_rcaud, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (register_qt12, datasink,[('out_file', '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (register_qt12, get_voxRhipp, [('out_file', 'in_file')]),
                       (extract_rhipp, get_voxRhipp, [('out_file', 'mask')]),
                       (register_qt12, get_voxLhipp, [('out_file', 'in_file')]),
                       (extract_lhipp, get_voxLhipp, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRamyg, [('out_file', 'in_file')]),
                       (extract_ramyg, get_voxRamyg, [('out_file', 'mask')]),
                       (register_qt12, get_voxLamyg, [('out_file', 'in_file')]),
                       (extract_lamyg, get_voxLamyg, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRput, [('out_file', 'in_file')]),
                       (extract_rput, get_voxRput, [('out_file', 'mask')]),
                       (register_qt12, get_voxLput, [('out_file', 'in_file')]),
                       (extract_lput, get_voxLput, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRcaud, [('out_file', 'in_file')]),
                       (extract_rcaud, get_voxRcaud, [('out_file', 'mask')]),
                       (register_qt12, get_voxLcaud, [('out_file', 'in_file')]),
                       (extract_lcaud, get_voxLcaud, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRnacc, [('out_file', 'in_file')]),
                       (extract_rnacc, get_voxRnacc, [('out_file', 'mask')]),
                       (register_qt12, get_voxLnacc, [('out_file', 'in_file')]),
                       (extract_lnacc, get_voxLnacc, [('out_file', 'mask')]),
                       
                       (get_voxLhipp, datasink, [('out_file', '2_Extracted_Stats')]),
                       (get_voxRhipp, datasink, [('out_file', '2_Extracted_Stats.@par')]),
                       (get_voxLamyg, datasink, [('out_file', '2_Extracted_Stats.@par.@par')]),
                       (get_voxRamyg, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par')]),
                       (get_voxLput, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par')]),
                       (get_voxRput, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par')]),
                       (get_voxLnacc, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par')]),
                       (get_voxRnacc, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par')]),
                       (get_voxLcaud, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (get_voxRcaud, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par.@par.@par')])
                      ])

hippdata_flow.base_dir = workflow_dir
hippdata_flow.write_graph(graph2use='flat')
preproc = hippdata_flow.run('MultiProc', plugin_args={'n_procs': 4})

In [ ]:
subject_list